# 4 new financial diaries

In this notebook we will add finacial diaries data for 4 new countries. 

- [Here](https://basecamp.com/1756858/projects/12871501/messages/83841824) the info of the new datasets.

- [Here](https://github.com/Vizzuality/i2i#importing-financial-diaries-data-csv) some notes from Ivan about what we should take into account.

- [Here](http://i2ifacility.org/data-portal/IND/financial-diaries) is how they look like in the website.

- [Here](https://basecamp.com/1756858/projects/14166276/messages/72313579) we have additional information and the first datasets we uploaded. 

## Data tables

The data is stored in two different files, one for `Households` (_temp_results_hh.csv) and the other one for `Individuals` (_temp_results_mem.csv).

The csvs must be properly formatted with the columns containing descriptive data (eg: `category_name`, `subcategory`), these columns will vary depending on the type of data being imported (`houshold_transactions` or `household_member_transactions`). The values must have a date header (eg: `2017-12`) and it's content must be 10 values separated by a colon: `160:40:0:160:null:0:null:null:null:null`.

Values can be `0`, `null`, or a `float`.

The 10 values correspond respectively to:

```
total_transaction_value
avg_value
min_value
max_value
rolling_balance
business_expenses
withdrawals
deposits
new_borrowing
repayment
```

### Data structure

**Households columns**

|Old data| New data |
|:---|:---|
|project_name  | project_name
|household_name| household_name
|category_type | cat_type
|category_name | 
|subcategory   | cat
|num_accounts  |
|num_members   |
|num_adults    |


**Individuals columns**

|Old data| New data |
|:---|:---|
|project_name         | project_name    
|household_name       | household_name
|person_code          | person_code
|gender               | gender
|age                  | age
|relationship_to_head | relationship_to_head
|employed             | employed
|status               | status
|category_type        | cat_type
|category_name        | 
|subcategory          | cat
|num_accounts         |


**Extra columns in new data**

- country
- project_id
- household_id
- respid

**Dates range**

|Old data| New data |
|:---|:---|
|2011-08 - 2016-12  | 2015-04 - 2017-08 |

In [221]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import math

**Households table**

In [42]:
households = pd.read_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/CSV/_temp_results_hh.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13,14,15,16,17,18,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,59,60,61,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
households.head(1)

,project_name,household_name,category_type,category_name,subcategory,num_accounts,num_members,num_adults,2011-08,2011-09,...,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
0,Kenya Financial Diaries,KNBOK01,NaN,ALL,NaN,7,4,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
print(list(households.columns)[:9])

['project_name', 'household_name', 'category_type', 'category_name', 'subcategory', 'num_accounts', 'num_members', 'num_adults', '2011-08']


In [45]:
list(households.columns)[-1]

'2016-12'

**Individuals table**

In [46]:
individuals = pd.read_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/CSV/_temp_results_mem.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17,18,19,20,21,22,39,42,43,44,45,46,47,48,49,50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,68,69,70,71,72,73,74,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
individuals.head(1)

,project_name,household_name,person_code,gender,age,relationship_to_head,employed,status,category_type,category_name,...,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
0,Kenya Financial Diaries,KNBOK01,P1,female,40,01=Head of this household,07=Casual work,in,NaN,Financial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
print(list(individuals.columns)[:13])

['project_name', 'household_name', 'person_code', 'gender', 'age', 'relationship_to_head', 'employed', 'status', 'category_type', 'category_name', 'subcategory', 'num_accounts', '2011-08']


In [49]:
list(households.columns)[-1]

'2016-12'

In [50]:
households['category_type'].unique()

array([nan, 'credits', 'expense', 'income', 'savings', 'insurance'],
      dtype=object)

In [51]:
households['category_name'].unique()

array(['ALL', 'Financial', 'Allowances, donations, and gifts', 'Clothing',
       'Communications', 'Education', 'Food/beverage', 'Home Care',
       'Multi-purpose shop', 'Personal Care Goods and Services',
       'Shelter and Utilites', 'Transportation', 'Operational Income',
       'Care items', 'Care services', 'Entertainment', 'Home care',
       'Shelter and utilites', 'Taxes, Bribes and Misc. fees',
       'Taxes and Fees', 'Financial Instruments', 'Taxes and fees',
       'Income and Resources', 'Non-employment categories',
       'Business Expenses', 'Financial Flows', 'Medical expenses',
       'Care items & services',
       'Religious expenses (funerals, weddings, etc)',
       'Taxes and fees, bribes', 'Multi-purpose shopping'], dtype=object)

In [52]:
households[(households['category_type'] == 'credits') & (households['project_name'] == 'India Financial Diaries')]['subcategory'].unique()

array([nan, 'Credit at store', 'Friends and family: borrowing',
       'Moneylender Borrowing', 'Joint Liability Loan',
       'Arrears owed by respondent', 'Business loan'], dtype=object)

In [53]:
households[(households['category_type'] == 'credits') & (households['project_name'] == 'India Financial Diaries')]['category_name'].unique()

array(['ALL', 'Financial'], dtype=object)

In [54]:
df = households[(households['category_type'] == 'credits') & (households['project_name'] == 'India Financial Diaries')]
df[['project_name', 'category_type', 'category_name', 'subcategory', 'num_accounts', 'num_members', 'num_adults', '2013-03']].head()

,project_name,category_type,category_name,subcategory,num_accounts,num_members,num_adults,2013-03
45,India Financial Diaries,credits,ALL,NaN,4,3,2,-18000.00:100:100.00:100.00:600000.00:0.00:nul...
50,India Financial Diaries,credits,Financial,Credit at store,2,3,2,NaN
51,India Financial Diaries,credits,Financial,Friends and family: borrowing,1,3,2,-18000.00:100:100.00:100.00:null:0.00:null:nul...
52,India Financial Diaries,credits,Financial,Moneylender Borrowing,1,3,2,null:null:null:null:null:0.00:null:null:null:null
83,India Financial Diaries,credits,ALL,NaN,2,6,2,-81000.00:338:300.00:375.00:936000.00:0.00:nul...


In [55]:
households['project_name'].unique()

array(['Kenya Financial Diaries', 'India Financial Diaries',
       'Mexico Financial Diaries', 'South Africa GAFIS',
       'US Financial Diaries', 'Kopo Kopo', 'Smallholder Diaries',
       'Smallholders Mozambique', 'Smallholders Pakistan',
       'Smallholders Tanzania', 'UJ South Africa',
       'Harambee South Africa'], dtype=object)

In [59]:
bangladesh['cat_type'].unique()

array(['expense', 'income', 'na', 'credits', 'saving'], dtype=object)

In [61]:
bangladesh[bangladesh['cat_type'] == "expense"]['cat'].unique()

array(['agriculture', 'event', 'food', 'health',
       'household or personal item', 'housing', 'leisure',
       'miscellaneous', 'service', 'transfer', 'transport',
       'communication', 'no transactions', 'education', 'fuel'],
      dtype=object)

In [62]:
bangladesh['cat'].unique()

array(['agriculture', 'employment', 'event', 'food', 'health',
       'household or personal item', 'housing', 'leisure', 'loan given',
       'loan repayment given', 'miscellaneous', 'sale of goods/services',
       'saving deposit', 'saving withdrawal', 'service', 'transfer',
       'transport', 'communication', 'credit purchase', 'loan received',
       'loan repayment received', 'no transactions', 'education', 'dps',
       'fuel', 'interest given', 'pf contribution', 'interest received',
       'jewelry mortgage'], dtype=object)

In [65]:
cambodia['cat_type'].unique()

array(['expense', 'income', 'na', 'credits', 'saving'], dtype=object)

In [68]:
salvador['cat_type'].unique()

array(['expense', 'na', 'income', 'credits', 'saving'], dtype=object)

In [71]:
guatemala['cat_type'].unique()

array(['expense', 'na', 'credits', 'saving', 'income'], dtype=object)

## MFO Diaries Data 

### Bangladesh

In [72]:
bangladesh = pd.read_csv('./data/MFO_DIaries_Data_June_2019/Bangladesh.csv')
bangladesh.drop(columns='Unnamed: 0', inplace=True)
bangladesh.head(1)

,country,project_name,project_id,household_name,household_id,person_code,gender,age,relationship_to_head,employed,...,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08
0,Bangladesh,Garment Worker Diaries,GWD,GWD-116,116,116.2,female,24,spouse of the head of household,factory worker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-320.0:320.0:320.0:320.0:320.0:0.0:0.0:0.0:0.0...,-150.0:150.0:150.0:150.0:470.0:0.0:0.0:0.0:0.0...,NaN


In [73]:
print(list(bangladesh.columns)[:15])

['country', 'project_name', 'project_id', 'household_name', 'household_id', 'person_code', 'gender', 'age', 'relationship_to_head', 'employed', 'status', 'respid', 'cat', 'cat_type', '2015-11']


In [74]:
bangladesh['cat_type'].unique()

array(['expense', 'income', 'na', 'credits', 'saving'], dtype=object)

In [75]:
list(bangladesh.columns)[-1]

'2017-08'

In [76]:
bangladesh.rename(columns={'cat_type': 'category_type', 'cat': 'category_name'}, inplace=True)
bangladesh['project_name'] = 'Bangladesh Financial Diaries'
bangladesh_hh = bangladesh.drop(columns=['person_code', 'gender', 'age', 
                                         'relationship_to_head', 'employed', 'status',
                                         'country', 'project_id', 'household_id', 'respid'
                                        ]
                               )
bangladesh_mem = bangladesh.drop(columns=['country', 'project_id', 'household_id', 'respid'])

**Create all column**

In [465]:
def all_year_values(df, year):
    nsubcat = len(df)
    array = []
    for i in range(nsubcat):
        results = df[year].iloc[i]
        if (type(results) == float) and math.isnan(results):
            a = np.empty((10))
            a[:] = np.nan
            array.append(list(a))
        else:
            sub_array = []
            for x in results.split(':'):
                if x == 'null':
                    sub_array.append(np.nan)
                else:
                    sub_array.append(float(x))
            array.append(sub_array)  
        
    array = np.array(array)

    result = []
    for i in range(10):
        if np.isnan(array[:,i]).all() == True:
            result.append('null')
        else:
            if i == 0:
                result.append(str(np.nansum(array[:,i])))
            if i == 1:
                result.append(str(np.nanmean(array[:,i])))
            if i == 2:
                result.append(str(np.nanmin(array[:,i])))
            if i == 3:
                result.append(str(np.nanmax(array[:,i])))
            if i > 3:
                result.append(str(np.nansum(array[:,i])))
                
    if result == ['null']*10:
        return np.nan
    else:
        return ":".join(str(x) for x in result)   

In [485]:
def create_ALL_row(df):
    df_new = pd.DataFrame(columns=list(df.columns))

    hh_list = list(df['household_name'].unique())

    for hh in hh_list:
        df_i = df[df['household_name'] == hh]
        df_i0 = df_i.iloc[0]
        a = np.empty((len(df_i.columns)))
        a[:] = np.nan
        df_all = pd.DataFrame([list(a)], columns=list(df_i.columns))
        df_all['project_name'] = df_i0['project_name']
        df_all['household_name'] = df_i0['household_name']
        df_all['category_name'] = 'ALL'
        
        other_columns = ['person_code', 'gender', 'age', 'relationship_to_head', 'employed', 'status']
        
        for column in other_columns:
            if column in list(df_i.columns):
                df_all[column] = df_i0[column]
                

        columns = list(df_i.columns)
        year_columns = [i for i in columns if i.startswith('20')]

        for year in year_columns:
            df_all[year] = all_year_values(df_i, year)

        df_new = pd.concat([df_new, df_all, df_i])
        
    return df_new

In [490]:
bangladesh_hh = create_ALL_row(bangladesh_hh)
bangladesh_mem = create_ALL_row(bangladesh_mem)

### Cambodia

In [77]:
cambodia = pd.read_csv('./data/MFO_DIaries_Data_June_2019/Cambodia.csv')
cambodia.drop(columns='Unnamed: 0', inplace=True)
cambodia.head(1)

,country,project_name,project_id,household_name,household_id,person_code,gender,age,relationship_to_head,employed,...,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06
0,Cambodia,Garment Worker Diaries,GWD,GWD-128,128,128.2,female,23,NaN,factory worker,...,-12600.0:6300.0:8400.0:4200.0:21000.0:0.0:0.0:...,-4200.0:4200.0:4200.0:4200.0:25200.0:0.0:0.0:0...,-12600.0:6300.0:8400.0:4200.0:37800.0:0.0:0.0:...,-12600.0:4200.0:4200.0:4200.0:50400.0:0.0:0.0:...,-25200.0:5040.0:8400.0:4200.0:75600.0:0.0:0.0:...,-4200.0:4200.0:4200.0:4200.0:79800.0:0.0:0.0:0...,-12500.0:4166.666666666667:4200.0:4100.0:92300...,-12600.0:6300.0:8400.0:4200.0:104900.0:0.0:0.0...,-16800.0:4200.0:4200.0:4200.0:121700.0:0.0:0.0...,-16700.0:4175.0:4200.0:4100.0:138400.0:0.0:0.0...


In [78]:
print(list(cambodia.columns)[:15])

['country', 'project_name', 'project_id', 'household_name', 'household_id', 'person_code', 'gender', 'age', 'relationship_to_head', 'employed', 'status', 'respid', 'cat', 'cat_type', '2016-07']


In [79]:
cambodia['project_name'].unique()

array(['Garment Worker Diaries'], dtype=object)

In [80]:
list(cambodia.columns)[-1]

'2017-06'

In [81]:
cambodia.rename(columns={'cat_type': 'category_type', 'cat': 'category_name'}, inplace=True)
cambodia['project_name'] = 'Cambodia Financial Diaries'
cambodia_hh = cambodia.drop(columns=['person_code', 'gender', 'age', 
                                         'relationship_to_head', 'employed', 'status',
                                         'country', 'project_id', 'household_id', 'respid'
                                        ]
                               )
cambodia_mem = cambodia.drop(columns=['country', 'project_id', 'household_id', 'respid'])

**Create all column**

In [491]:
cambodia_hh = create_ALL_row(cambodia_hh)
cambodia_mem = create_ALL_row(cambodia_mem)

### El Salvador

In [82]:
salvador = pd.read_csv('./data/MFO_DIaries_Data_June_2019/El_Salvador.csv')
salvador.drop(columns='Unnamed: 0', inplace=True)
salvador.head(1)

,country,project_name,project_id,household_name,household_id,person_code,gender,age,relationship_to_head,employed,...,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04
0,El Salvador,Savings Group Diaries,SGD,SGD-101,101,101.1,2,23,head of household,20.0,...,-1.0:1.0:1.0:1.0:9.5:0.0:0.0:0.0:0.0:0.0,NaN,NaN,NaN,NaN,-2.0:2.0:2.0:2.0:11.5:0.0:0.0:0.0:0.0:0.0,NaN,NaN,NaN,-1.0:1.0:1.0:1.0:12.5:0.0:0.0:0.0:0.0:0.0


In [83]:
print(list(salvador.columns)[:15])

['country', 'project_name', 'project_id', 'household_name', 'household_id', 'person_code', 'gender', 'age', 'relationship_to_head', 'employed', 'status', 'respid', 'cat', 'cat_type', '2015-04']


In [84]:
list(salvador.columns)[-1]

'2016-04'

Change gender categories

In [85]:
salvador.loc[salvador['gender'] == 1, 'gender'] = 'male'
salvador.loc[salvador['gender'] == 2, 'gender'] = 'female'

In [86]:
salvador.rename(columns={'cat_type': 'category_type', 'cat': 'category_name'}, inplace=True)
salvador['project_name'] = 'El Salvador Financial Diaries'
salvador_hh = salvador.drop(columns=['person_code', 'gender', 'age', 
                                         'relationship_to_head', 'employed', 'status',
                                         'country', 'project_id', 'household_id', 'respid'
                                        ]
                               )
salvador_mem = salvador.drop(columns=['country', 'project_id', 'household_id', 'respid'])

**Create all column**

In [492]:
salvador_hh = create_ALL_row(salvador_hh)
salvador_mem = create_ALL_row(salvador_mem)

### Guatemala

In [87]:
guatemala = pd.read_csv('./data/MFO_DIaries_Data_June_2019/Guatemala.csv')
guatemala.drop(columns='Unnamed: 0', inplace=True)
guatemala.head(1)

,country,project_name,project_id,household_name,household_id,person_code,gender,age,relationship_to_head,employed,...,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05
0,Guatemala,Savings Group Diaries,SGD,SGD-401,401,401.1,2,32,spouse,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-36.0:36.0:36.0:36.0:36.0:0.0:0.0:0.0:0.0:0.0,NaN,NaN


In [88]:
print(list(guatemala.columns)[:15])

['country', 'project_name', 'project_id', 'household_name', 'household_id', 'person_code', 'gender', 'age', 'relationship_to_head', 'employed', 'status', 'respid', 'cat', 'cat_type', '2015-05']


In [89]:
list(guatemala.columns)[-1]

'2016-05'

Change gender categories

In [90]:
guatemala.loc[guatemala['gender'] == 1, 'gender'] = 'male'
guatemala.loc[guatemala['gender'] == 2, 'gender'] = 'female'

In [91]:
guatemala.rename(columns={'cat_type': 'category_type', 'cat': 'category_name'}, inplace=True)
guatemala['project_name'] = 'Guatemala Financial Diaries'
guatemala_hh = guatemala.drop(columns=['person_code', 'gender', 'age', 
                                         'relationship_to_head', 'employed', 'status',
                                         'country', 'project_id', 'household_id', 'respid'
                                        ]
                               )
guatemala_mem = guatemala.drop(columns=['country', 'project_id', 'household_id', 'respid'])

**Create all column**

In [493]:
guatemala_hh = create_ALL_row(guatemala_hh)
guatemala_mem = create_ALL_row(guatemala_mem)

## New tables

In [494]:
hh_dic = {'old' : households,
          'bangladesh': bangladesh_hh,
          'cambodia': cambodia_hh,
          'salvador': salvador_hh,
          'guatemala': guatemala_hh
         }
mem_dic = {'old' : individuals,
           'bangladesh': bangladesh_mem,
           'cambodia': cambodia_mem,
           'salvador': salvador_mem,
           'guatemala': guatemala_mem
          }

In [495]:
hh_columns = list(households.columns)[:8]
mem_columns = list(individuals.columns)[:12]
years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

n = 0
for year in years:
    for month in months:
        if n > 6:
            hh_columns.append(year+'-'+month)
            mem_columns.append(year+'-'+month)
        n += 1
        
hh_new =  pd.DataFrame(columns=hh_columns)
mem_new = pd.DataFrame(columns=mem_columns)

for df_hh in hh_dic.values():
    hh_iter =  pd.DataFrame(columns=hh_columns)
    for column in list(df_hh.columns):
        hh_iter[column] = df_hh[column]
        
    hh_new = pd.concat([hh_new, hh_iter])
    
for df_mem in mem_dic.values():
    mem_iter =  pd.DataFrame(columns=mem_columns)
    for column in list(df_mem.columns):
        mem_iter[column] = df_mem[column]
        
    mem_new = pd.concat([mem_new, mem_iter])

In [496]:
hh_new.head()

,project_name,household_name,category_type,category_name,subcategory,num_accounts,num_members,num_adults,2011-08,2011-09,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
0,Kenya Financial Diaries,KNBOK01,NaN,ALL,NaN,7,4,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kenya Financial Diaries,KNBOK01,credits,ALL,NaN,7,4,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kenya Financial Diaries,KNBOK01,expense,ALL,NaN,0,4,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kenya Financial Diaries,KNBOK01,income,ALL,NaN,16,4,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kenya Financial Diaries,KNBOK01,savings,ALL,NaN,10,4,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [497]:
hh_new['project_name'].unique()

array(['Kenya Financial Diaries', 'India Financial Diaries',
       'Mexico Financial Diaries', 'South Africa GAFIS',
       'US Financial Diaries', 'Kopo Kopo', 'Smallholder Diaries',
       'Smallholders Mozambique', 'Smallholders Pakistan',
       'Smallholders Tanzania', 'UJ South Africa',
       'Harambee South Africa', 'Bangladesh Financial Diaries',
       'Cambodia Financial Diaries', 'El Salvador Financial Diaries',
       'Guatemala Financial Diaries'], dtype=object)

**Save new tables**

In [504]:
mem_new.to_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/CSV_new/_temp_results_mem.csv')
hh_new.to_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/CSV_new/_temp_results_hh.csv')

Only new countries

In [37]:
hh = hh_new[hh_new['project_name'].isin(['Bangladesh Financial Diaries',
       'Cambodia Financial Diaries', 'El Salvador Financial Diaries',
       'Guatemala Financial Diaries'])]

In [38]:
mem = mem_new[mem_new['project_name'].isin(['Bangladesh Financial Diaries',
       'Cambodia Financial Diaries', 'El Salvador Financial Diaries',
       'Guatemala Financial Diaries'])]

In [42]:
mem.to_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/CSV_new/_temp_results_mem.csv')
hh.to_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/CSV_new/_temp_results_hh.csv')

### New data structure

We split the table into two.

#### Households table

- **Single table structure**:
    - `project_name`, `household_name`, `category_type`, `category_name`, `subcategory`, `num_accounts`, `num_members`, `num_adults`, `2011-08`, `2011-09` ...
    
The data at date header (eg: `2017-12`) is structures as 10 values separated by a colon: `160:40:0:160:null:0:null:null:null:null`.

Values can be `0`, `null`, or a `float`.

The 10 values correspond respectively to:

```
total_transaction_value
avg_value
min_value
max_value
rolling_balance
business_expenses
withdrawals
deposits
new_borrowing
repayment
```

- **Two table structure**:
    - **household_transactions**: `id`, `project_name`, `household_name`, `category_type`, `category_name`, `subcategory`, `num_accounts`, `num_members`, `num_adults`, `total_income`
    - **household_transaction_histories**: `id`, `household_transaction_id`, `value`, `month`, `year`, `total_transaction_value`, `avg_value`, `min_value`, `max_value`, `rolling_balance`, `business_expenses`, `withdrawals`, `deposits`, `new_borrowing`, `repayment`, `date`
    
The `household_transactions` and `household_transaction_histories` tables are related with the `id` and `household_transaction_id` ids, respectively.

#### Household members table

- **Single table structure**:
    - `project_name`, `household_name`, `person_code`, `gender`, `age`, `relationship_to_head`, `employed`, `status`, `category_type`, `category_name`, `subcategory`, `num_accounts`, `2011-08`, `2011-09` ...
    
The data at date header (eg: `2017-12`) is structures as 10 values separated by a colon: `160:40:0:160:null:0:null:null:null:null`.

Values can be `0`, `null`, or a `float`.

The 10 values correspond respectively to:

```
total_transaction_value
avg_value
min_value
max_value
rolling_balance
business_expenses
withdrawals
deposits
new_borrowing
repayment
```

- **Two table structure**:
    - **household_member_transactions**: `id`, `project_name`, `household_name`, `person_code`, `gender`,
       `relationship_to_head`, `employed`, `status`, `category_type`,
       `category_name`, `subcategory`, `age`, `num_accounts`, `total_income`
    - **household_member_transaction_histories**: `id`, `household_transaction_id`, `value`, `month`, `year`, `total_transaction_value`, `avg_value`, `min_value`, `max_value`, `rolling_balance`, `business_expenses`, `withdrawals`, `deposits`, `new_borrowing`, `repayment`, `date`
    
The `household_transactions` and `household_transaction_histories` tables are related with the `id` and `household_transaction_id` ids, respectively.

In [ ]:
hh_new.head(1)

In [ ]:
mem_new.head(1)

In [ ]:
household_trans = pd.read_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_transactions.csv')
household_trans.drop(columns=['created_at', 'updated_at'], inplace=True)
household_trans_hist = pd.read_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_transaction_histories.csv')
household_trans_hist.drop(columns=['created_at', 'updated_at'], inplace=True)

In [ ]:
household_trans.head(1)

In [ ]:
household_trans_hist.head(1)

In [ ]:
household_member_trans = pd.read_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_member_transactions.csv')
household_member_trans.drop(columns=['created_at', 'updated_at'], inplace=True)
household_member_trans_hist = pd.read_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_member_transaction_histories.csv')
household_member_trans_hist.drop(columns=['created_at', 'updated_at'], inplace=True)

In [ ]:
household_member_trans.head(1)

In [ ]:
household_member_trans_hist.head(1)

**We transform the table for the new countries only**

In [ ]:
hh = hh_new[hh_new['project_name'].isin(['Bangladesh Financial Diaries',
       'Cambodia Financial Diaries', 'El Salvador Financial Diaries',
       'Guatemala Financial Diaries'])]

In [ ]:
hh.head(2)

In [ ]:
mem = mem_new[mem_new['project_name'].isin(['Bangladesh Financial Diaries',
       'Cambodia Financial Diaries', 'El Salvador Financial Diaries',
       'Guatemala Financial Diaries'])]

In [ ]:
mem.head(2)

In [ ]:
household_trans = pd.read_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_transactions.csv')

### Split table

**Household Transactions**

Add `id` column

In [ ]:
hh['id'] = np.arange(len(hh))+household_trans['id'].max()+1
mem['id'] = np.arange(len(mem))+household_member_trans['id'].max()+1

In [ ]:
household_member_trans.columns

In [ ]:
mem.columns

In [ ]:
hh_trans = hh[['id','project_name', 'household_name', 'category_type',
       'category_name', 'subcategory', 'num_accounts', 'num_members',
       'num_adults']]

mem_trans = mem[['id', 'project_name', 'household_name', 'person_code', 'gender',
       'relationship_to_head', 'employed', 'status', 'category_type',
       'category_name', 'subcategory', 'age', 'num_accounts']]

Add `total_income` column

In [ ]:
hh_trans['total_income'] = np.nan
mem_trans['total_income'] = np.nan

In [ ]:
hh_trans.head(1)

In [ ]:
mem_trans.head(1)

**Household Transaction Histories**

In [ ]:
years = ['2015', '2016', '2017']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

dates = []
for year in years:
    for month in months:
        dates.append(year+'-'+month)

In [ ]:
hh_trans_hist = pd.DataFrame(columns=['household_transaction_id', 'value', 'month', 'year',
       'total_transaction_value', 'avg_value', 'min_value', 'max_value',
       'rolling_balance', 'business_expenses', 'withdrawals', 'deposits',
       'new_borrowing', 'repayment', 'date'])

params = ['total_transaction_value', 'avg_value', 'min_value', 'max_value', 'rolling_balance', 'business_expenses', 'withdrawals', 'deposits', 'new_borrowing', 'repayment']

hh_trans_hist_row = pd.DataFrame(columns=['household_transaction_id', 'value', 'month', 'year',
       'total_transaction_value', 'avg_value', 'min_value', 'max_value',
       'rolling_balance', 'business_expenses', 'withdrawals', 'deposits',
       'new_borrowing', 'repayment', 'date'])
for row in tqdm(range(len(hh))):
    for year in dates:
        
        hh_trans_hist_row['household_transaction_id'] = [hh['id'].iloc[row]]
        hh_trans_hist_row['value'] = [hh[year].iloc[row]]
        hh_trans_hist_row['month'] = [int(year.split('-')[1])]
        hh_trans_hist_row['year'] = [year.split('-')[0]]    
        
        if str(hh[year].iloc[row]) != 'nan':
            values = [round(float(i),1) for i in hh[year].iloc[row].split(':')]
        else:
            values = np.empty(10); values[:] = np.nan
            
        for n, param in enumerate(params):
            hh_trans_hist_row[param] = [values[n]]
            
        hh_trans_hist_row['date'] = year+'-01 00:00:00'
        
        hh_trans_hist = pd.concat([hh_trans_hist, hh_trans_hist_row])
            

In [ ]:
mem_trans_hist = pd.DataFrame(columns=['household_transaction_id', 'value', 'month', 'year',
       'total_transaction_value', 'avg_value', 'min_value', 'max_value',
       'rolling_balance', 'business_expenses', 'withdrawals', 'deposits',
       'new_borrowing', 'repayment', 'date'])

params = ['total_transaction_value', 'avg_value', 'min_value', 'max_value', 'rolling_balance', 'business_expenses', 'withdrawals', 'deposits', 'new_borrowing', 'repayment']

mem_trans_hist_row = pd.DataFrame(columns=['household_transaction_id', 'value', 'month', 'year',
       'total_transaction_value', 'avg_value', 'min_value', 'max_value',
       'rolling_balance', 'business_expenses', 'withdrawals', 'deposits',
       'new_borrowing', 'repayment', 'date'])
for row in tqdm(range(len(hh))):
    for year in dates:
        
        mem_trans_hist_row['household_transaction_id'] = [mem['id'].iloc[row]]
        mem_trans_hist_row['value'] = [mem[year].iloc[row]]
        mem_trans_hist_row['month'] = [int(year.split('-')[1])]
        mem_trans_hist_row['year'] = [year.split('-')[0]]    
        
        if str(mem[year].iloc[row]) != 'nan':
            values = [round(float(i),1) for i in mem[year].iloc[row].split(':')]
        else:
            values = np.empty(10); values[:] = np.nan
            
        for n, param in enumerate(params):
            mem_trans_hist_row[param] = [values[n]]
            
        mem_trans_hist_row['date'] = year+'-01 00:00:00'
        
        mem_trans_hist = pd.concat([mem_trans_hist, mem_trans_hist_row])

**Concatenate DataFrames**

In [ ]:
household_trans_new = pd.concat([household_trans, hh_trans])
household_trans_new.to_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_transactions_new.csv')

In [ ]:
household_trans_hist.drop(columns='id', inplace=True)
household_trans_hist_new = pd.concat([household_trans_hist, hh_trans_hist])
household_trans_hist_new['id'] = np.arange(len(household_trans_hist_new))
household_trans_hist_new.to_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_transactions_histories_new.csv')

In [ ]:
household_member_trans_new = pd.concat([household_member_trans, mem_trans])
household_member_trans_new.to_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_member_transactions_new.csv')

In [ ]:
household_member_trans_hist.drop(columns='id', inplace=True)
household_member_trans_hist_new = pd.concat([household_member_trans_hist, mem_trans_hist])
household_member_trans_hist_new['id'] = np.arange(len(household_member_trans_hist_new))
household_member_trans_hist_new.to_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/Financial_Diaries/household_member_transactions_histories_new.csv')

**Merge tables**

In [ ]:
hh_trans = pd.read_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/household_new/household_transactions_new.csv')
hh_trans.drop(columns='Unnamed: 0', inplace=True)
hh_trans.rename(columns={'id': 'household_transaction_id'}, inplace=True)

hh_trans_hist = pd.read_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/household_new/household_transactions_histories_new.csv')
hh_trans_hist.drop(columns=['Unnamed: 0'], inplace=True)

household = pd.merge(left=hh_trans, right=hh_trans_hist, on= 'household_transaction_id')
household.to_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/household_new/household_transactions_merged.csv')

In [19]:
hh_trans = pd.read_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/household_new/household_member_transactions_new.csv')
hh_trans.drop(columns='Unnamed: 0', inplace=True)
hh_trans.rename(columns={'id': 'household_member_transaction_id'}, inplace=True)

hh_trans_hist = pd.read_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/household_new/household_member_transactions_histories_new.csv')
hh_trans_hist.drop(columns=['Unnamed: 0'], inplace=True)

household = pd.merge(left=hh_trans, right=hh_trans_hist, on= 'household_member_transaction_id')
household.to_csv('/Volumes/MacBook HD/data/i2i/Data Iker/Financial_Diaries/household_new/household_member_transactions_merged.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
